# Парсинг численности учеников и других данных 

Используемый ресурс: https://my.edu.kz/ru/school

Сначала код ищет бин, затем на карте находит маркер, нажимает на него и потом парсит данные 

In [3]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [4]:
df = pd.read_excel("Atyrau_schools.xlsx")

In [5]:
#если бины даны не в формате 12 цифр, то можно добавить нули перед бином таким способом
id_list = df['business_id'].astype(str).str.zfill(12)

In [6]:
id_list

0      000240003249
1      000240003259
2      000240003279
3      000340000654
4      000340002353
           ...     
236    980840004073
237    990440001829
238    990440001859
239    990440001899
240    990440004764
Name: business_id, Length: 241, dtype: object

In [11]:
#запускаем окно браузера, ОБЯЗАТЕЛЬНО расширьте его на весь экран
driver = webdriver.Chrome() 

In [12]:
# Создаем пустой список для данных
data = []

In [13]:
# Функция для попытки клика по маркеру, так как часто маркер остается не замеченным
def click_marker(driver):
    try:
        # Попытка найти маркер и кликнуть по нему
        marker = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'ymaps-2-1-79-islets_icon-caption'))
        )
        ActionChains(driver).move_to_element(marker).click(marker).perform()
        return True
    except (NoSuchElementException, TimeoutException):
        # Если элемент не найден, возвращаем False
        return False

In [14]:
# Основной цикл для перебора Бинов
for business_id in id_list:
    try:
        # Формирование URL и его открытие
        unique_url = f'https://my.edu.kz/ru/schools?q={business_id}'
        driver.get(unique_url)
        # Переменная для контроля цикла
        marker_clicked = False
        # Счетчик попыток клика по маркеру
        attempt_count = 0

        # Повторяем попытки, пока маркер не будет успешно кликнут или пока не исчерпаны попытки
        while not marker_clicked and attempt_count < 3:
            marker_clicked = click_marker(driver)
            if not marker_clicked:
                time.sleep(2)  # Небольшая задержка перед следующей попыткой
                attempt_count += 1
        # Если маркер не найден после N попыток, записываем "нет данных" и переходим к следующему ID
        if not marker_clicked:
            data.append({"business_id": business_id, "some_data": "Нет данных"})
            continue  # Пропускаем оставшуюся часть цикла и переходим к следующему business_id
                       
        # После успешного клика по маркеру, ожидаем появления ссылки и кликаем по ней
        link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '&id=')]"))
        )
        link.click()

        # Получаем исходный код страницы
        html = driver.page_source
        # Парсим страницу с помощью BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        # Находим все элементы с классом 'col-12 col-sm-6'
        elements = soup.find_all(class_='col-12 col-sm-6')
        
        # Проверка наличия нужного элемента и его обработка
        if len(elements) > 17:
            seventeenth_element_text = elements[17].text.strip()
            # Добавление данных в список
            data.append({"business_id": business_id, "some_data": seventeenth_element_text})
        else:
                print("Не найдено достаточное количество элементов.")
    except Exception as e:
        print(f"Произошла ошибка: {e}")
    finally:
        # Задержка для избежания слишком частых запросов
        time.sleep(4)

# Закрываем браузер после завершения всех запросов
driver.quit()

Не найдено достаточное количество элементов.
Произошла ошибка: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C8025E42+3538674]
	(No symbol) [0x00007FF7C7C44C02]
	(No symbol) [0x00007FF7C7AF5AEB]
	(No symbol) [0x00007FF7C7B3BF4E]
	(No symbol) [0x00007FF7C7B3C0CC]
	(No symbol) [0x00007FF7C7B7E477]
	(No symbol) [0x00007FF7C7B5F0EF]
	(No symbol) [0x00007FF7C7B7BDE3]
	(No symbol) [0x00007FF7C7B5EE53]
	(No symbol) [0x00007FF7C7B2F514]
	(No symbol) [0x00007FF7C7B30631]
	GetHandleVerifier [0x00007FF7C8056CAD+3738973]
	GetHandleVerifier [0x00007FF7C80AC506+4089270]
	GetHandleVerifier [0x00007FF7C80A4823+4057299]
	GetHandleVerifier [0x00007FF7C7D75C49+720121]
	(No symbol) [0x00007FF7C7C5126F]
	(No symbol) [0x00007FF7C7C4C304]
	(No symbol) [0x00007FF7C7C4C432]
	(No symbol) [0x00007FF7C7C3BD04]
	BaseThreadInitThunk [0x00007FFFEC88257D+29]
	RtlUserThreadStart [0x00007FFFEDDEAA58+40]

Произошла ошибка: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C8025E42+3538674]
	(No symbol) [0x00007F

In [15]:
# Создание датафрейма из собранных данных
df_results = pd.DataFrame(data)

In [16]:
data

[{'business_id': '000240003249', 'some_data': '535'},
 {'business_id': '000240003259', 'some_data': '327'},
 {'business_id': '000240003279', 'some_data': '377'},
 {'business_id': '000340000654', 'some_data': '51'},
 {'business_id': '000340002353', 'some_data': '327'},
 {'business_id': '000340002581', 'some_data': '618'},
 {'business_id': '000340003252', 'some_data': '82'},
 {'business_id': '000340003292', 'some_data': 'Нет данных'},
 {'business_id': '000340003301', 'some_data': '376'},
 {'business_id': '000340003311', 'some_data': '73'},
 {'business_id': '000340003321', 'some_data': '132'},
 {'business_id': '000340003331', 'some_data': '320'},
 {'business_id': '000340003341', 'some_data': '333'},
 {'business_id': '000340003351', 'some_data': '105'},
 {'business_id': '000340003361', 'some_data': '399'},
 {'business_id': '000340003410', 'some_data': '245'},
 {'business_id': '000340003470', 'some_data': '239'},
 {'business_id': '000340003490', 'some_data': '358'},
 {'business_id': '000340

In [17]:
# Сохранение датафрейма в Excel файл
excel_filename = "schools_data.xlsx"
df_results.to_excel(excel_filename, index=False)

print(f"Файл успешно сохранен: {excel_filename}")

Файл успешно сохранен: schools_data.xlsx


# Дальше можно воспользовать кодом: объединение экселек по бинам

# Чтоб найти какой N элемент, можно использовать следующий код:

Например номер элемента, где хранится значение Скорости интернета по договору: 

In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://my.edu.kz/ru/schools?q=160740012565&id=1194'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Найти элементы с классом 'col-12 col-sm-6'
elements = soup.find_all(class_='col-12 col-sm-6')

# Получить второй элемент из списка
if len(elements) > 23:
    second_element = elements[23]
    print(second_element.text.strip())
else:
    print("Второй элемент не найден")


10 Мбит/с
